In [1]:
import numpy as np

In [2]:
import checklist
from checklist.test_suite import TestSuite
import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
from transformers import pipeline

In [4]:
model = pipeline("question-answering", model="./standard_squad_model/", device=0)

In [5]:
suite_path = '../../../checklist/release_data/squad/squad_suite.pkl'
suite = TestSuite.from_file(suite_path)

In [6]:
def predconfs(context_question_pairs):
    preds = []
    confs = []
    for c, q in context_question_pairs:
        try:
            p = model(question=q, context=c, truncation=True, )
        except:
            print('Failed', q)
            preds.append(' ')
            confs.append(1)
        preds.append(p['answer'])
        confs.append(p['score'])
    return preds, np.array(confs)

In [7]:
suite.run(predconfs, overwrite=True, n=100)   # for quicker testing 

Running A is COMP than B. Who is more / less COMP?
Predicting 200 examples
Running Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?
Predicting 1200 examples
Running size, shape, age, color
Predicting 400 examples
Running Profession vs nationality
Predicting 1000 examples
Running Animal vs Vehicle
Predicting 400 examples
Running Animal vs Vehicle v2
Predicting 400 examples
Running Synonyms
Predicting 400 examples
Running A is COMP than B. Who is antonym(COMP)? B
Predicting 400 examples
Running A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.
Predicting 1600 examples
Running Question typo
Predicting 200 examples
Running Question contractions
Predicting 201 examples
Running Add random sentence to context
Predicting 300 examples
Running Change name everywhere
Predicting 1100 examples
Running Change location everywhere
Predicting 1100 examples
Running There was a change in profession
Predicting 200 exampl

In [8]:
def format_squad_with_context(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'C: %s\nQ: %s\n' % (c, q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

In [10]:
suite.summary(format_example_fn=format_squad_with_context)

Vocabulary

A is COMP than B. Who is more / less COMP?
Test cases:      494
Test cases run:  100
Fails (rate):    100 (100.0%)

Example fails:
C: Timothy is cleaner than Kelly.
Q: Who is less clean?
A: Kelly
P: Timothy


----
C: Jordan is smaller than Nathan.
Q: Who is less small?
A: Nathan
P: Jordan


----
C: Rebecca is taller than Samantha.
Q: Who is less tall?
A: Samantha
P: Rebecca


----


Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?
Test cases:      497
Test cases run:  100
Fails (rate):    100 (100.0%)

Example fails:
C: Anna is extremely pessimistic about the project. Eric is somewhat pessimistic about the project.
Q: Who is least pessimistic about the project?
A: Eric
P: Anna

C: Anna is extremely pessimistic about the project. Eric is pessimistic about the project.
Q: Who is least pessimistic about the project?
A: Eric
P: Anna

C: Anna is pessimistic about the project. Eric is somewhat pessimistic about the project.
Q: Who is least pessimistic ab

In [11]:
suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'A is COMP than B. Wh…

# SQUAD RESULT:

In [ ]:
{'eval_exact_match': 78.240302743614, 'eval_f1': 85.98667304511686}
